In [1]:
import json
import pandas as pd
from matplotlib import pyplot as plt
from scipy import stats
import numpy as np
import base64
import requests

In [2]:
keys = pd.read_csv("Spotify-Keys_10-18-22.txt", header = 0, sep = ",")
appid = keys['Client_ID'][0]
appsecret = keys['Client_Secret'][0]
client_cred = base64.b64encode(str(appid + ":" + appsecret).encode('ascii'))
header = {'Authorization': 'Basic {}'.format(client_cred.decode('ascii'))}
payload = {'grant_type' : 'client_credentials'}
access_token_url = "https://accounts.spotify.com/api/token"
response = requests.post(access_token_url, headers = header, data = payload)
session_token = response.json()['access_token']
session_header = {'Authorization': 'Bearer {}'.format(response.json()['access_token'])}

In [16]:
#Logic: Vinyl Days
days_id = '0RTHMVF27KRMHIGnF7iLru'
album_tracks_endpoint = "https://api.spotify.com/v1/albums/{}/tracks".format(days_id)
days_tracks = requests.get(album_tracks_endpoint, headers = session_header)
print(days_tracks.status_code)
days_dict = json.loads(days_tracks.text)
days_track_ids = [x['id'] for x in days_dict['items']]
days_track_strings = ','.join(str(x) for x in days_track_ids)
print(days_track_strings)
track_features_endpoint = "https://api.spotify.com/v1/audio-features?ids={}".format(days_track_strings)

200
0xjp9Li9jmlykJNrIlAnqe,7tQpKKS4Jis0pCotemeLdz,6FoyBTuvyX2YFw4QW1e6DK,6yiCFprNLeKraW5RsYUxiE,6nI37NimdoCoRvPwxXZaMt,1LFDDwqjZtSzjXWIYXbY5O,0IBHTkmEQCLGHJaWs1tuEg,3RTCSJf6MKrEXNjws0G6Ou,5ZK1XF3s2KrFw63RBf6ciX,0XfjQgy46bjlJ4bOOenNS6,4VlMA2A8JRZnSI4Owdbnau,7xeic9Mig0gIkpMlMV4G0i,0mvWpmMlAcxHkjLu9yXdoS,6Rn5ruN2hIMqx9U8JNVHGz,4Fs7O2jQCQMgU3uka3Qeci,7Bi5CtEcDIduA53TdPHMLF,4XvqNiFFUcS50jjVz3enH7,2Gt5hPrCYs1VxXFRzCFF2N,6CDIIpcpMfERHtf4MOdxr7,13cRWsPNUmbgcSMPgOibVJ


In [17]:
days_features = requests.get(track_features_endpoint, headers = session_header)
days_dict = json.loads(days_features.text)
days_dict['audio_features'][0]

{'danceability': 0.461,
 'energy': 0.937,
 'key': 6,
 'loudness': -5.065,
 'mode': 1,
 'speechiness': 0.468,
 'acousticness': 0.112,
 'instrumentalness': 0.000964,
 'liveness': 0.111,
 'valence': 0.547,
 'tempo': 164.73,
 'type': 'audio_features',
 'id': '0xjp9Li9jmlykJNrIlAnqe',
 'uri': 'spotify:track:0xjp9Li9jmlykJNrIlAnqe',
 'track_href': 'https://api.spotify.com/v1/tracks/0xjp9Li9jmlykJNrIlAnqe',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/0xjp9Li9jmlykJNrIlAnqe',
 'duration_ms': 44333,
 'time_signature': 5}

In [18]:
days_df = pd.DataFrame(days_dict['audio_features'])
days_df.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.461,0.937,6,-5.065,1,0.468,0.1120,0.000964,0.111,0.547,164.730,audio_features,0xjp9Li9jmlykJNrIlAnqe,spotify:track:0xjp9Li9jmlykJNrIlAnqe,https://api.spotify.com/v1/tracks/0xjp9Li9jmly...,https://api.spotify.com/v1/audio-analysis/0xjp...,44333,5
1,0.708,0.980,6,-3.122,1,0.251,0.1180,0.000000,0.614,0.585,96.879,audio_features,7tQpKKS4Jis0pCotemeLdz,spotify:track:7tQpKKS4Jis0pCotemeLdz,https://api.spotify.com/v1/tracks/7tQpKKS4Jis0...,https://api.spotify.com/v1/audio-analysis/7tQp...,124974,4
2,0.659,0.884,6,-3.710,0,0.283,0.5390,0.000000,0.260,0.826,93.367,audio_features,6FoyBTuvyX2YFw4QW1e6DK,spotify:track:6FoyBTuvyX2YFw4QW1e6DK,https://api.spotify.com/v1/tracks/6FoyBTuvyX2Y...,https://api.spotify.com/v1/audio-analysis/6Foy...,108387,4
3,0.364,0.745,9,-5.842,0,0.240,0.0413,0.000000,0.442,0.517,90.951,audio_features,6yiCFprNLeKraW5RsYUxiE,spotify:track:6yiCFprNLeKraW5RsYUxiE,https://api.spotify.com/v1/tracks/6yiCFprNLeKr...,https://api.spotify.com/v1/audio-analysis/6yiC...,148597,3
4,0.561,0.947,7,-3.476,1,0.258,0.2570,0.938000,0.272,0.664,97.896,audio_features,6nI37NimdoCoRvPwxXZaMt,spotify:track:6nI37NimdoCoRvPwxXZaMt,https://api.spotify.com/v1/tracks/6nI37NimdoCo...,https://api.spotify.com/v1/audio-analysis/6nI3...,19606,4


In [19]:
#Drake's Her Loss
loss_id = '5MS3MvWHJ3lOZPLiMxzOU6'
album_tracks_endpoint = "https://api.spotify.com/v1/albums/{}/tracks".format(loss_id)
loss_tracks = requests.get(album_tracks_endpoint, headers = session_header)
print(loss_tracks.status_code)
loss_dict = json.loads(loss_tracks.text)
loss_track_ids = [x['id'] for x in loss_dict['items']]
loss_track_strings = ','.join(str(x) for x in loss_track_ids)
print(loss_track_strings)
track_features_endpoint = "https://api.spotify.com/v1/audio-features?ids={}".format(loss_track_strings)

200
1bDbXMyjaUIooNwFE9wn0N,46s57QULU02Voy0Kup6UEb,34tz0eDhGuFErIuW3q4mPX,0wshkEEcJUQU33RSRBb5dv,7l2nxyx7IkBX5orhkALg0V,2ZL7WZcjuYKi1KUDtp4kCC,0sSRLXxknVTQDStgU1NqpY,4Flfb4fGscN9kXPOduQLrv,7GeTsDIc5ykNB6lORO6Cee,2KLwPaRDOB87XOYAT2fgxh,45pUWUeEWGoSdH6UF162G8,410TZrK18uRjtsTunG14cl,2ZlABhxMMa43COmZ97kKsJ,29YW2xXlo6Pc6M0SBTCXYN,1MTXYvJ9TSqg9x6WPCDx2n,31bsuKDOzFGzBAoXxtnAJm


In [20]:
loss_features = requests.get(track_features_endpoint, headers = session_header)
loss_dict = json.loads(loss_features.text)
loss_dict['audio_features'][0]

{'danceability': 0.561,
 'energy': 0.52,
 'key': 11,
 'loudness': -9.342,
 'mode': 0,
 'speechiness': 0.244,
 'acousticness': 0.0503,
 'instrumentalness': 1.86e-06,
 'liveness': 0.355,
 'valence': 0.424,
 'tempo': 153.15,
 'type': 'audio_features',
 'id': '1bDbXMyjaUIooNwFE9wn0N',
 'uri': 'spotify:track:1bDbXMyjaUIooNwFE9wn0N',
 'track_href': 'https://api.spotify.com/v1/tracks/1bDbXMyjaUIooNwFE9wn0N',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/1bDbXMyjaUIooNwFE9wn0N',
 'duration_ms': 239360,
 'time_signature': 3}

In [21]:
loss_df = pd.DataFrame(loss_dict['audio_features'])
loss_df.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.561,0.520,11,-9.342,0,0.244,0.05030,0.000002,0.3550,0.424,153.150,audio_features,1bDbXMyjaUIooNwFE9wn0N,spotify:track:1bDbXMyjaUIooNwFE9wn0N,https://api.spotify.com/v1/tracks/1bDbXMyjaUIo...,https://api.spotify.com/v1/audio-analysis/1bDb...,239360,3
1,0.908,0.546,8,-10.491,0,0.323,0.01270,0.000000,0.0707,0.225,131.016,audio_features,46s57QULU02Voy0Kup6UEb,spotify:track:46s57QULU02Voy0Kup6UEb,https://api.spotify.com/v1/tracks/46s57QULU02V...,https://api.spotify.com/v1/audio-analysis/46s5...,170502,4
2,0.841,0.358,9,-8.368,1,0.591,0.02010,0.000000,0.3900,0.329,158.014,audio_features,34tz0eDhGuFErIuW3q4mPX,spotify:track:34tz0eDhGuFErIuW3q4mPX,https://api.spotify.com/v1/tracks/34tz0eDhGuFE...,https://api.spotify.com/v1/audio-analysis/34tz...,261544,4
3,0.849,0.433,5,-8.434,0,0.316,0.04070,0.000000,0.3880,0.395,142.110,audio_features,0wshkEEcJUQU33RSRBb5dv,spotify:track:0wshkEEcJUQU33RSRBb5dv,https://api.spotify.com/v1/tracks/0wshkEEcJUQU...,https://api.spotify.com/v1/audio-analysis/0wsh...,152962,4
4,0.934,0.614,5,-7.384,1,0.198,0.00274,0.000000,0.1230,0.616,144.045,audio_features,7l2nxyx7IkBX5orhkALg0V,spotify:track:7l2nxyx7IkBX5orhkALg0V,https://api.spotify.com/v1/tracks/7l2nxyx7IkBX...,https://api.spotify.com/v1/audio-analysis/7l2n...,160821,4


In [22]:
#Nas' King's Disease 3
king_id = '5MS3MvWHJ3lOZPLiMxzOU6'
album_tracks_endpoint = "https://api.spotify.com/v1/albums/{}/tracks".format(king_id)
king_tracks = requests.get(album_tracks_endpoint, headers = session_header)
print(king_tracks.status_code)
king_dict = json.loads(king_tracks.text)
king_track_ids = [x['id'] for x in king_dict['items']]
king_track_strings = ','.join(str(x) for x in king_track_ids)
print(king_track_strings)
track_features_endpoint = "https://api.spotify.com/v1/audio-features?ids={}".format(king_track_strings)

200
1bDbXMyjaUIooNwFE9wn0N,46s57QULU02Voy0Kup6UEb,34tz0eDhGuFErIuW3q4mPX,0wshkEEcJUQU33RSRBb5dv,7l2nxyx7IkBX5orhkALg0V,2ZL7WZcjuYKi1KUDtp4kCC,0sSRLXxknVTQDStgU1NqpY,4Flfb4fGscN9kXPOduQLrv,7GeTsDIc5ykNB6lORO6Cee,2KLwPaRDOB87XOYAT2fgxh,45pUWUeEWGoSdH6UF162G8,410TZrK18uRjtsTunG14cl,2ZlABhxMMa43COmZ97kKsJ,29YW2xXlo6Pc6M0SBTCXYN,1MTXYvJ9TSqg9x6WPCDx2n,31bsuKDOzFGzBAoXxtnAJm


In [23]:
king_features = requests.get(track_features_endpoint, headers = session_header)
king_dict = json.loads(king_features.text)
king_dict['audio_features'][0]

{'danceability': 0.561,
 'energy': 0.52,
 'key': 11,
 'loudness': -9.342,
 'mode': 0,
 'speechiness': 0.244,
 'acousticness': 0.0503,
 'instrumentalness': 1.86e-06,
 'liveness': 0.355,
 'valence': 0.424,
 'tempo': 153.15,
 'type': 'audio_features',
 'id': '1bDbXMyjaUIooNwFE9wn0N',
 'uri': 'spotify:track:1bDbXMyjaUIooNwFE9wn0N',
 'track_href': 'https://api.spotify.com/v1/tracks/1bDbXMyjaUIooNwFE9wn0N',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/1bDbXMyjaUIooNwFE9wn0N',
 'duration_ms': 239360,
 'time_signature': 3}

In [24]:
king_df = pd.DataFrame(king_dict['audio_features'])
king_df.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.561,0.520,11,-9.342,0,0.244,0.05030,0.000002,0.3550,0.424,153.150,audio_features,1bDbXMyjaUIooNwFE9wn0N,spotify:track:1bDbXMyjaUIooNwFE9wn0N,https://api.spotify.com/v1/tracks/1bDbXMyjaUIo...,https://api.spotify.com/v1/audio-analysis/1bDb...,239360,3
1,0.908,0.546,8,-10.491,0,0.323,0.01270,0.000000,0.0707,0.225,131.016,audio_features,46s57QULU02Voy0Kup6UEb,spotify:track:46s57QULU02Voy0Kup6UEb,https://api.spotify.com/v1/tracks/46s57QULU02V...,https://api.spotify.com/v1/audio-analysis/46s5...,170502,4
2,0.841,0.358,9,-8.368,1,0.591,0.02010,0.000000,0.3900,0.329,158.014,audio_features,34tz0eDhGuFErIuW3q4mPX,spotify:track:34tz0eDhGuFErIuW3q4mPX,https://api.spotify.com/v1/tracks/34tz0eDhGuFE...,https://api.spotify.com/v1/audio-analysis/34tz...,261544,4
3,0.849,0.433,5,-8.434,0,0.316,0.04070,0.000000,0.3880,0.395,142.110,audio_features,0wshkEEcJUQU33RSRBb5dv,spotify:track:0wshkEEcJUQU33RSRBb5dv,https://api.spotify.com/v1/tracks/0wshkEEcJUQU...,https://api.spotify.com/v1/audio-analysis/0wsh...,152962,4
4,0.934,0.614,5,-7.384,1,0.198,0.00274,0.000000,0.1230,0.616,144.045,audio_features,7l2nxyx7IkBX5orhkALg0V,spotify:track:7l2nxyx7IkBX5orhkALg0V,https://api.spotify.com/v1/tracks/7l2nxyx7IkBX...,https://api.spotify.com/v1/audio-analysis/7l2n...,160821,4
